# Backpropagation Through Single *Sampled* Logit.
We test the ability to backpropagate through a single logit.  If you only allow gradients through a single logit, then at a given training iteration, you will only impact one column of your weight matrix, that is, weights contributing to the value of that logit. 

In [4]:
import tensorflow as tf
from tensorflow.contrib.distributions import Categorical
tf.reset_default_graph()

input_dim = 3
hidden_dim = 5
output_dim = 3
lr = 1e-1
num_iterations = 50
print_every = 1

x = tf.fill([1, input_dim], 1.)
y = tf.fill([1, output_dim], 1.)


with tf.name_scope('Model'):
    W_gen = tf.Variable(tf.random_uniform([input_dim, hidden_dim]), name = 'W_gen')
    logits = tf.matmul(x, W_gen)

    # Sample through single logit.
    categorical = Categorical(logits)
    index = tf.stop_gradient(categorical.sample())
#     index = tf.stop_gradient(Categorical(logits).sample()
#     index = tf.squeeze(sample_op)
#     index = tf.constant(0, dtype=tf.int32)
    one_hot = tf.one_hot(index, hidden_dim, dtype = tf.float32)
    logits = logits * one_hot

    W_dis = tf.Variable(tf.random_uniform([hidden_dim, output_dim]), name = 'W_dis')
    output = tf.matmul(logits, W_dis)

        
with tf.name_scope('Loss'):
    loss_op = tf.reduce_mean(tf.squared_difference(output, y))

with tf.name_scope('Train'):
    train_vars = [W_gen]
    train_op = tf.train.AdamOptimizer(lr).minimize(loss_op, var_list = train_vars)
    
    
with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)

    for i in xrange(num_iterations):
        if i % print_every == 0:
            print('Loss at iteration %d: %f' % (i, sess.run(loss_op)))
            print('Sample: [%d]' % sess.run(index))
            print sess.run(W_gen)
        sess.run(train_op)
    print sess.run(output)


Loss at iteration 0: 0.595851
Sample: [3]
[[ 0.83569515  0.13769221  0.77196145  0.58878052  0.30522072]
 [ 0.78347039  0.50974977  0.25244844  0.84535742  0.30451775]
 [ 0.25584829  0.79547215  0.14775825  0.79268515  0.19243336]]
Loss at iteration 1: 0.079502
Sample: [1]
[[ 0.73569524  0.13769221  0.77196145  0.58878052  0.30522072]
 [ 0.68347049  0.50974977  0.25244844  0.84535742  0.30451775]
 [ 0.15584838  0.79547215  0.14775825  0.79268515  0.19243336]]
Loss at iteration 2: 0.266778
Sample: [1]
[[ 0.66868973  0.21210557  0.77196145  0.58878052  0.30522072]
 [ 0.61646497  0.58416313  0.25244844  0.84535742  0.30451775]
 [ 0.08884285  0.8698855   0.14775825  0.79268515  0.19243336]]
Loss at iteration 3: 0.266778
Sample: [1]
[[ 0.61689448  0.26962709  0.83584219  0.58878052  0.30522072]
 [ 0.56466973  0.64168465  0.31632921  0.84535742  0.30451775]
 [ 0.03704758  0.92740703  0.21163902  0.79268515  0.19243336]]
Loss at iteration 4: 0.212614
Sample: [1]
[[ 0.57446676  0.34147394  0.8

### Analysis
Training, even for the most trivial task imaginable, is very volatile.  This does not appear to be a plausible way to proceed.

### TensorFlow Bug
Mathematical impossibility. For a drawn sample $i$, columns $\neq i$ of the generator weight matrix $W_{gen}$ are updating! This only occurs when I'm using Categorical.  This is filed as [Issue 4074](https://github.com/tensorflow/tensorflow/issues/4074).


# Distribution vs. One-Hot


## Forward Propagation.

In [76]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.distributions import Categorical
tf.reset_default_graph()

input_dim = 3
output_dim = 1
num_samples = 250

x = tf.constant([[0.2, 0.3, 0.5]], dtype = tf.float32) 

with tf.name_scope('dist_input'):
    W_dis = tf.Variable(tf.random_uniform([input_dim, output_dim]), name = 'W_dis')
    output = tf.matmul(x, W_dis)


with tf.name_scope('one_hot_input'):
    tf.get_variable_scope().reuse_variables()
    outputs = []
    
    for i in xrange(num_samples):
        sample_op = Categorical(x).sample(n=1)
        index = tf.squeeze(sample_op)
        x = tf.one_hot(index, input_dim, dtype = tf.float32)
        x = tf.reshape(x, [1, input_dim])
        outputs.append(tf.matmul(x, W_dis))
    
        
with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    print('Distribution: %f' %  sess.run(tf.squeeze(output)))
    print('One-hot: %f' % np.average(sess.run(outputs)))


Distribution: 0.800210
One-hot: 0.799524
